In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_list = []

csv_files = [f for f in os.listdir('data/')] 
for file in csv_files:
    file_path = os.path.join('data/', file)
    data_list.append(pd.read_csv(file_path))

df = pd.concat(data_list, ignore_index=True)

df[df['class'] == 'back-slouch'] #Grabs only a specific class.

    

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z32,v32,x33,y33,z33,v33,neck_tilt,shoulder_tilt,mouth_tilt,eye_tilt
0,back-slouch,0.509448,0.524600,-0.522874,0.999766,0.522115,0.491173,-0.499200,0.999721,0.528078,...,0.017744,0.025120,0.437206,1.868234,0.106335,0.011936,30.468780,112.674779,139.763642,142.459910
1,back-slouch,0.514575,0.521808,-0.545932,0.999773,0.526927,0.488849,-0.514747,0.999734,0.534232,...,0.158668,0.028173,0.440444,1.970423,0.146236,0.015577,30.317279,115.427690,128.360529,130.256377
2,back-slouch,0.515132,0.520802,-0.558694,0.999788,0.528085,0.488747,-0.520031,0.999754,0.535698,...,0.152009,0.026906,0.443615,2.081633,0.108856,0.015270,30.678226,113.655348,128.445598,130.600115
3,back-slouch,0.517845,0.519300,-0.536412,0.999783,0.531239,0.487632,-0.492078,0.999755,0.538949,...,0.182745,0.026613,0.453008,2.142330,0.090229,0.016016,30.861230,112.990364,131.319567,130.669615
4,back-slouch,0.519888,0.519069,-0.510512,0.999778,0.533118,0.487201,-0.465184,0.999753,0.540787,...,0.154047,0.024961,0.456965,2.190049,0.033289,0.015050,31.096813,111.697687,130.607744,130.688743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,back-slouch,0.516997,0.582927,-0.786696,0.999635,0.533911,0.529648,-0.743954,0.999519,0.546062,...,0.146419,0.007864,0.526228,2.630286,0.011694,0.004360,25.916642,98.081265,93.581431,92.887480
1521,back-slouch,0.518752,0.582860,-0.828009,0.999583,0.536297,0.529527,-0.784796,0.999448,0.548107,...,0.142695,0.008064,0.525429,2.607839,0.015237,0.004493,26.226720,98.865955,93.659231,94.261023
1522,back-slouch,0.520127,0.582674,-0.826925,0.999503,0.537705,0.529236,-0.782005,0.999345,0.549334,...,0.157013,0.008125,0.527924,2.610840,0.042601,0.004570,26.323485,98.975024,93.736933,94.366873
1523,back-slouch,0.521544,0.581971,-0.801416,0.999500,0.538959,0.528550,-0.754149,0.999343,0.550353,...,0.118418,0.008151,0.548240,2.613820,-0.000210,0.004498,26.495344,98.986534,95.967361,94.807027


In [2]:
X = df.drop('class', axis=1) #Features (no class col)
y = df['class'] #Target value (class col)


In [3]:
#Create train/ test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# Train Classification Model

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [5]:
"""pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=200)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier(max_iter=200)),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

#Train every model (4) on the data.
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model"""

pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())
model = pipeline.fit(X_train, y_train)


In [6]:
model.predict(X_test)

array(['side-lean', 'forward-slouch', 'forward-slouch', ..., 'upright',
       'upright', 'side-lean'], dtype=object)

# Evaluate & Serialise Models

In [7]:
from sklearn.metrics import accuracy_score
import pickle

In [8]:
"""for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))"""

yhat = model.predict(X_test)
print(f"rf: {accuracy_score(y_test, yhat)}")

rf: 0.9941783540619211


In [10]:
"""# SAVE MODEL #
with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)"""

with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(model, f)

# Make Detections w/ Model

In [11]:
# RELOAD INTO NOTEBOOK #
with open('pose_detection.pkl', 'rb') as f:
    model = pickle.load(f)

